In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Change working directory to be current folder
import os
os.chdir('/content/drive/MyDrive/ExerciseScoring')
!ls

Mounted at /content/drive
 best_model_ex2.h5    Kimore_ex2	      'pretrain model'
'best model ex4'      Kimore_ex4	       README.md
 best_model_ex4.h5   'KIMORE Preprocessed'     rehabilitation_ex2.json
 Data		      LICENSE		       rehabilitation_ex4.json
 demo.ipynb	      person2_ex1_NPr_NP.mp4   requirements.txt
 demo_predict.ipynb   person2_ex1_NPr_P.mp4    train.ipynb
 demo.py	      person2_ex1_Pr_NP.mp4    train.py
 Figure		      person2_ex1_Pr_P.mp4
 GCN		      predict.ipynb


In [ ]:
!pip install tensorflow_addons
!pip install mediapipe

from google.colab.patches import cv2_imshow
import cv2
import mediapipe as mp
import time
import os
import csv
import pandas as pd
import imutils

import numpy as np
import pandas as pd
import joblib
import tensorflow as tf
from tensorflow.keras.layers import concatenate, Flatten, Dropout, Dense, Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import *
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from GCN.sgcn_lstm import Sgcn_Lstm

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 32.8 MB 1.2 MB/s 


In [ ]:

index_Head=1   # no orientation (no.1, bp 0, nose)
index_Shoulder_Left=46 # (no.2, bp 11, left_shoulder)
index_Elbow_Left=54 # (no.3, bp 13, left_elbow)
index_Wrist_Left=62 # (no.4, bp 15, left_wrist)
index_Hand_Left=70 # (no.5, bp 18, left_pinky)
index_Shoulder_Right=50 # (no.6, bp 12, right_shoulder)
index_Elbow_Right=58 # (no.7, bp 14, elbow_right)
index_Wrist_Right=66 # (no.8, bp 16, right_wrist)
index_Hand_Right=74 # (no.9, bp 18, right_pinky)
index_Hip_Left=94 # (no.10, bp 23, left_hip)
index_Knee_Left=102 # (no.11, bp 25, left_knee)
index_Ankle_Left=110 # (no.12, bp 27, left_ankle)
index_Foot_Left=126  # no orientation # (no.13, bp 31, left_foot_index)  
index_Hip_Right=94 # (no.14, bp 24, right_hip)
index_Knee_Right=106 # (no.15, bp 26, right_knee)
index_Ankle_Right=114 # (no.16, bp 28, right_ankle)
index_Foot_Right=130   # no orientation # (no.17, bp 32, right_foot_index)

index_Tip_Left=78     # no orientation (no.18, bp 19, left_index)
index_Thumb_Left=86   # no orientation (no.19, bp 21, left_thumb)
index_Tip_Right=82    # no orientation (no.20, bp 20, right_index)
index_Thumb_Right=90  # no orientation (no.21, bp 22, right_index)

body_part = [index_Head, index_Shoulder_Left, index_Elbow_Left, index_Wrist_Left, index_Hand_Left, index_Shoulder_Right, index_Elbow_Right, index_Wrist_Right, index_Hand_Right, index_Hip_Left, index_Knee_Left, index_Ankle_Left, index_Foot_Left, index_Hip_Right, index_Knee_Right, index_Ankle_Right, index_Ankle_Right, index_Tip_Left, index_Thumb_Left, index_Tip_Right, index_Thumb_Right
]

In [ ]:
class PoseDetector:

    def __init__(self, mode = False, modelComplexity = 1, smooth=True,  enable_segmentation=False, smooth_segmentation=True, detectionCon = 0.65, trackCon = 0.65):

        self.mode = mode
        self.modelComplexity = modelComplexity
        self.smooth = smooth
        self. enable_segmentation =  enable_segmentation
        self. smooth_segmentation =  smooth_segmentation
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.modelComplexity, self.smooth, self.enable_segmentation, self.smooth_segmentation, self.detectionCon, self.trackCon)

    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        #print(results.pose_landmarks)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks, self.mpPose.POSE_CONNECTIONS)

        return img

    def getPosition(self, img, draw=True):
        lmList= []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                #print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmList.append(id)
                lmList.append(lm.x)
                lmList.append(lm.y)
                lmList.append(lm.z)
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return lmList

def main():
    cap = cv2.VideoCapture(vidname)
    pTime = 0
    detector = PoseDetector()
    while cap.isOpened():
        success, img = cap.read()
        img = detector.findPose(img)
        if ImList:
            lmList = detector.getPosition(img)
            #print(lmList)

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        cv2.imshow("Image", img)
        # Wait until a key is pressed.
        # Retreive the ASCII code of the key pressed
        k = cv2.waitKey(1) 
        
        # Check if 'ESC' is pressed.
        if k == ord('b'):
            
            # Break the loop.
            break
     
    # Release the VideoCapture object.
    cap.release()
     
    # Close the windows.
    cv2.destroyAllWindows()

In [ ]:
def predict(input):
    """Load the data"""
    x = input.iloc[:,:].values
    #label = pd.read_csv(labels, header = None).iloc[:,:].values
    batch_size = 1
    num_timestep = 100
    num_channel = 3
    num_joints = 21

    """Load the scaling object"""

    sc_y = joblib.load("Data/sc_y.save")
    
    X = np.zeros((x.shape[0],num_joints*num_channel)).astype('float32')
    for row in range(x.shape[0]):
        counter = 0
        for parts in body_part:
            for i in range(num_channel):
                X[row, counter+i] = x[row, parts+i]
            counter += num_channel 
    
    X = StandardScaler().fit_transform(X)          
    
    X_ = np.zeros((batch_size, num_timestep, num_joints, num_channel))
    
    for batch in range(X_.shape[0]):
        for timestep in range(X_.shape[1]):
            for node in range(X_.shape[2]):
                for channel in range(X_.shape[3]):
                    X_[batch,timestep,node,channel] = X[timestep+(batch*num_timestep),channel+(node*num_channel)]
               
    X = X_
    
    """Load the model and weights"""
    #change exercise model here
    with open('rehabilitation_ex2.json', 'r') as f:
        model_json = f.read()
    
    model = tf.keras.models.model_from_json(model_json, custom_objects={'tf': tf})
    #change exercise weights here
    model.load_weights("best_model_ex2.h5")
    
    y_pred = model.predict(X)
    y_pred = sc_y.inverse_transform(y_pred) 
    print("predicted value:", y_pred[0,0])
    score = y_pred[0,0]
    return score    

In [ ]:
vidname = 'person2_ex1_Pr_P.mp4' #enter path to video here!
cap = cv2.VideoCapture(vidname)
pTime = 0
detector = PoseDetector()
joints = []


while cap.isOpened():
    success, img = cap.read()
    if success:
        img = detector.findPose(img)
        lmList = detector.getPosition(img)
        if lmList:
            joints.append(lmList)
            #print(lmList)
    
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        print("Please wait, keypoint detection ongoing")
        cv2.putText(img, str(int(fps)), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 3)
        #cv2_imshow("Image", img)
        
        # Wait until a key is pressed.
        # Retreive the ASCII code of the key pressed
        k = cv2.waitKey(1) 
        
        # Check if 'ESC' is pressed.
        if k == ord('b'):
            
            # Break the loop.
            break
    
    else:
        print("Keypoint detection completed!")
        break
 
# Release the VideoCapture object.
cap.release()
 
# Close the windows.
cv2.destroyAllWindows()

print('done')
fullstep = len(joints)
print('Total number of frames: ' + str(fullstep))
ystep = fullstep//100
numstep = (ystep) * 100

joints_100 = [] #joints_100 is rows that is multiple of 100
for i in range(numstep):
    timestep = i
    #print(timestep)
    joints_100.append(joints[timestep])

df = pd.DataFrame(joints_100)

print(df)
score = predict(df)

Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing
Please wait, keypoint detection ongoing


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


predicted value: 41.612846


In [ ]:
print(score)

41.612846
